### #TODEV
- [ ] breakpoint functionality/reading from ODV files
- [ ] sprof import
- [ ] br import
- [ ] normalize column names between MBARI and nc files
- [ ] repeat of rmode graphs for multiple data versions for one float

        df_bd1 = load_Bnc( '/Users/madison.soden/BGCARGO_work/data/4903624/BD/BD4903624_001.nc',  ['DOXY','NITRATE','PH_IN_SITU_TOTAL','CHLA','BBP700'])
        cycle_nums, df_24_bd_map = get_df_map_Bnc( '/Users/madison.soden/BGCARGO_work/data/4903624/BD/',   ['DOXY','NITRATE','PH_IN_SITU_TOTAL','CHLA','BBP700'])
        preview_df(df_bd1)
        all_df = list(df_24_bd_map.values())

### set globals and paths

make_df_dict(): will import data from MatLab data directory in this format:
    f'{matlab_data_dir}{wmo}/QC/ODV{wmo}QC.TXT')
    f'{matlab_data_dir}{wmo}/ODV{wmo}.TXT')
1) make sure all ODV files are upto date
2) if you want to include multiple data versions for one float make sure the files are named with a wmo suffix
       E.g: wmo = "4903622_mod_ref_ph_980"
            expected amode file name:  {matlab_data_dir} + "4903622_mod_ref_ph_980/QC/ODV4903622_mod_ref_ph_980QC.TXT"
            expected rmode file name:  {matlab_data_dir} + "4903622_mod_ref_ph_980/ODV4903622_mod_ref_ph_980.TXT"
            (you can just copy the rmode ODV file and rename it)

In [32]:
matlab_data_dir = '/Users/madison.soden/Documents/MATLAB/ARGO_PROCESSING/DATA/ARGO_REPO/'
WMO_LIST = ["6999992","6999992_ph_950_980","4903904", "4903904_ph_950_980"]

In [50]:
line_colors_greens =  [ '#1aa000', '#25e200', '#79ff5e'] #greens 
line_colors_reds   =  [ '#b22222', '#e65c00', '#ff8c42'] #reds 
line_colors_blues  =  [ '#007bff', '#5d9eff', '#87c4ff'] #blues
line_colors_greys  =  [ '#494541', '#696969', '#9f9f9f'] #greyscale
#px.colors.qualitative.Set3
#px.colors.sequential.Viridis_r

wmo_to_colorscale = {  "4903622": "blues", 
                       "4903624": "greens",
                       "4903625": "reds",
                       "7901009": "oranges",
                       "2904010": "purples",
                       "2904011": "teal",
                       "6999992": "reds",
                       "6999992_ph_950_980": "blues",
                       "4903904": "oranges", 
                       "4903904_ph_950_980": "purples"
                       } #"greys", "mint", "peach", "purp", "brwnyl" https://plotly.com/python/builtin-colorscales/
                  

### Imports

In [34]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import glob
import io
import os
import netCDF4
import xarray as xr
import plotly.graph_objects as go
import pandas as pd
pd.options.mode.copy_on_write = True

### Function declarations

In [35]:
def get_color_scale_rgb( colorscale, max_profile_num):
    """
    Generate discrete color scale given a max profile number. Returns a list of rgb tumpels 07/23/25 M.S. 
    Not sure what "colortype='hex' is doing, if anything.
    """
    if (max_profile_num ==1): max_profile_num+=1 #to prevent dividing by zero
    #positions = [i / (max_profile_num - 1) for i in range(max_profile_num)]
    #colors = px.colors.sample_colorscale( colorscale, positions, colortype='rgb')
    colors = px.colors.sample_colorscale( colorscale, max_profile_num, low=0.4, high=0.8, colortype='rgb') 
    return colors #colors[::-1]for reversed scale #colors for non reverse scale

In [36]:
def get_netcdf_files(directory):
        # Get all files and directories in the specified directory
        files = os.listdir(directory)
        # Filter out directories, only keep files
        files = [f for f in files if os.path.isfile(os.path.join(directory, f))]
        return files

In [37]:
def xarray_netcdf_df( filename): 
    ds = xr.open_dataset(filename, drop_variables=["DATA_TYPE", "FORMAT_VERSION", "HANDBOOK_VERSION", "HISTORY_SOFTWARE", "HISTORY_SOFTWARE_RELEASE", 
                                                   "HISTORY_REFERENCE", "HISTORY_DATE", "REFERENCE_DATE_TIME", "DATE_CREATION", "DATE_UPDATE", "HISTORY_ACTION",
                                                   "HISTORY_QCTEST", "HISTORY_START_PRES", "HISTORY_STOP_PRES", "PI_NAME", "PROJECT_NAME", "SCIENTIFIC_CALIB_COEFFICIENT"
                                                   "SCIENTIFIC_CALIB_EQUATION", 'PLATFORM_TYPE', 'FLOAT_SERIAL_NO', 'FIRMWARE_VERSION', 'WMO_INST_TYPE', 'JULD', 
                                                   'JULD_QC', 'JULD_LOCATION', 'LATITUDE', 'LONGITUDE', 'POSITION_QC', 'POSITIONING_SYSTEM', 'VERTICAL_SAMPLING_SCHEME',
                                                   'DIRECTION', 'DATA_CENTRE', 'DC_REFERENCE', 'SCIENTIFIC_CALIB_EQUATION', 'SCIENTIFIC_CALIB_COEFFICIENT', 'SCIENTIFIC_CALIB_COMMENT', 
                                                   'SCIENTIFIC_CALIB_DATE', 'HISTORY_INSTITUTION', 'HISTORY_STEP', 'HISTORY_PARAMETER', 'HISTORY_PREVIOUS_VALUE'])
    df = ds.to_dataframe()
    df = df.reset_index()
    df1 = df[['PRES', 'DOXY_ADJUSTED', 'DOXY_ADJUSTED_QC', 'BBP700_ADJUSTED', 'BBP700_ADJUSTED_QC', 'CHLA_ADJUSTED', 'CHLA_ADJUSTED_QC', 
               'NITRATE_ADJUSTED', 'NITRATE_ADJUSTED_QC']].copy() #'DOXY_ADJUSTED_ERROR', 'BBP700_ADJUSTED_ERROR', 'CHLA_ADJUSTED_ERROR', 'NITRATE_ADJUSTED_ERROR', 
    #print(list(df1.columns))
    df1 = df1.dropna(subset=["DOXY_ADJUSTED"])
    for c in ['DOXY_ADJUSTED_QC', 'BBP700_ADJUSTED_QC', 'CHLA_ADJUSTED_QC', 'NITRATE_ADJUSTED_QC']: 
        df1[c] = df1[c].apply(lambda x: x.decode('utf-8'))
    df1 = df1.drop_duplicates()
    df1.rename(columns={'BBP700_ADJUSTED':'BBP_700[1/m]', 'CHLA_ADJUSTED':'CHLA[mg/m^3]', 'NITRATE_ADJUSTED':'Nitrate[µmol/kg]', 'DOXY_ADJUSTED':'Oxygen[µmol/kg]',
                        'DOXY_ADJUSTED_QC':'OxygenQF', 'BBP700_ADJUSTED_QC':'BBPQF', 'CHLA_ADJUSTED_QC':'CHLAQF', 'NITRATE_ADJUSTED_QC':'NitrateQF', 'PRES':'Depth[m]' }, inplace=True)
    df1 = df1.reset_index(drop=True)
    return df1

In [38]:
def load_sprof( fileName, param_List):
    """
    Example function call: 
        df = load_sprof("/Users/madison.soden/Downloads/2904010_Sprof.nc", ["DOXY"])
    """
    #loading relevant sprof into pandas array
    ds = xr.open_dataset(fileName, drop_variables=["DATA_TYPE", "FORMAT_VERSION", "HANDBOOK_VERSION", "HISTORY_SOFTWARE", "HISTORY_SOFTWARE_RELEASE", 
                                                   "HISTORY_REFERENCE", "HISTORY_DATE", "REFERENCE_DATE_TIME", "DATE_CREATION", "DATE_UPDATE", "HISTORY_ACTION",
                                                   "HISTORY_QCTEST", "HISTORY_START_PRES", "HISTORY_STOP_PRES", "PI_NAME", "PROJECT_NAME", "SCIENTIFIC_CALIB_COEFFICIENT"
                                                   "SCIENTIFIC_CALIB_EQUATION", 'PLATFORM_TYPE', 'FLOAT_SERIAL_NO', 'FIRMWARE_VERSION', 'WMO_INST_TYPE', 'JULD', 
                                                   'JULD_QC', 'JULD_LOCATION', 'LATITUDE', 'LONGITUDE', 'POSITION_QC', 'POSITIONING_SYSTEM', 'VERTICAL_SAMPLING_SCHEME',
                                                   'DIRECTION', 'DATA_CENTRE', 'DC_REFERENCE', 'SCIENTIFIC_CALIB_EQUATION', 'SCIENTIFIC_CALIB_COEFFICIENT', 'SCIENTIFIC_CALIB_COMMENT', 
                                                   'SCIENTIFIC_CALIB_DATE', 'HISTORY_INSTITUTION', 'HISTORY_STEP', 'HISTORY_PARAMETER', 'HISTORY_PREVIOUS_VALUE'])

    
    keep_column_list = ['N_PROF', 'N_LEVELS', 'PRES', 'PRES_ADJUSTED']
    for p in param_List:
        keep_column_list.append(p)
        keep_column_list.append(p+"_QC")
        keep_column_list.append(p+"_ADJUSTED")
        keep_column_list.append(p+"_ADJUSTED_QC")
        keep_column_list.append(p+"_ADJUSTED_ERROR")
    
    # convert to pandas df
    df = ds.to_dataframe().reset_index()
        #print(df.columns)
    
    df_sprof = df[keep_column_list]
        #print(df_sprof.head(25))

    df_sprof = df_sprof.loc[df_sprof["DOXY"].notna()]
    
    return df_sprof

In [39]:
def det_skiprows_odvqc(filename):
    '''This function determines the number of header lines (starting with "//")
    in and ODV{floatid}QC.txt file with the given filename.
    Returns the number of header lines. 
    Hartmut Frenzel'''
    with open(filename, 'r') as f_in:
        lines = f_in.readlines()
    skiprows = 0
    nit_break_points = []
    for line in lines:
        if line[0:2] == '//':
            if line.startswith("//Nitrate"):
                Nit_coefs = line.split("\t")
                nit_break_points.append(int(Nit_coefs[1]))
            skiprows += 1
        else:
            break
    return skiprows, nit_break_points

In [40]:
def preview_df( df): 
    with pd.option_context(
        'display.max_rows', 200,
        'display.min_rows', 50,
        'display.max_columns', None): display(df)

In [41]:
def load_ODV( fileName):
    #loading relevant odvfile into pandas array
    skip, nit_break_points=det_skiprows_odvqc(fileName)
    df_ODV = pd.read_csv( fileName, sep='\t', skiprows=skip)
    
    df_ODV = df_ODV.reset_index(drop=True)
    df_ODV = df_ODV.rename(columns={"QF.1": "PressureQF", "QF.2": "TemperatureQF", "QF.3": "SalinityQF", 
                                    "QF.6": "OxygenQF", "QF.8": "NitrateQF", "QF.9": "CHLAQF", "QF.10": "BBPQF", "QF.12": "pH_QF"})
        # df_ODV = df_ODV.iloc[:, [ 1, 2, 3, 4, 8, 9, 10, 11, 12, 13, 16, 18, 19, 22, 23, 24, 25, 26, 27, 30, 31]]
        # pressure = 8, pressureQF = 9, Temp = 10, TempQF = 11, Salinity =12, SalinityQF= 13
        # depth = 16, Oxygen =18, oxygenQF = 19, nitrate = 22, nitrateQF = 23, 
        # Chl_a = 24, Chl_aQF = 25, BBP700 = 26, BBPQF = 27, pH = 30, pHQF = 31
    
        #foo = str(df_ODV.iloc[5, 'mon/day/yr']) + " " + str(df_ODV.iloc[5, 'hh:mm'])
    df_ODV.rename(columns={'b_bp700[1/m]':'BBP_700[1/m]'}, inplace=True)
    df_ODV.rename(columns={'Chl_a[mg/m^3]':'CHLA[mg/m^3]'}, inplace=True)
    df_ODV.rename(columns={'pHinsitu[Total]':'pH_Insitu_total'}, inplace=True)
    
    for i in df_ODV.index:
        df_ODV.loc[i, 'mon/day/yr'] = pd.to_datetime( (str(df_ODV['mon/day/yr'][i]) + " " + str(df_ODV['hh:mm'][i])), format='%m/%d/%Y %H:%M')
    
    for i in df_ODV.index: 
        if( df_ODV.loc[i, 'Nitrate[µmol/kg]'] == -10000000000.0 ):
            df_ODV.loc[i, 'Nitrate[µmol/kg]'] = None
            df_ODV.loc[i, 'NitrateQF'] = None
        if( df_ODV.loc[i, 'Oxygen[µmol/kg]'] == -10000000000.0 ):
            df_ODV.loc[i, 'Oxygen[µmol/kg]'] = None
            df_ODV.loc[i, 'OxygenQF'] = None
        if( df_ODV.loc[i, 'BBP_700[1/m]'] == -10000000000.0 ):
            df_ODV.loc[i, 'BBP_700[1/m]'] = None
            df_ODV.loc[i, 'BBPQF'] = None
        if( df_ODV.loc[i, 'pH_Insitu_total'] == -10000000000.0 ):
            df_ODV.loc[i, 'pH_Insitu_total'] = None
            df_ODV.loc[i, 'pH_QF'] = None
        if( df_ODV.loc[i, 'CHLA[mg/m^3]'] == -10000000000.0 ):
            df_ODV.loc[i, 'CHLA[mg/m^3]'] = None
            df_ODV.loc[i, 'CHLAQF'] = None
        if( df_ODV.loc[i, 'Salinity[pss]'] == -10000000000.0):
            df_ODV.loc[i, 'Salinity[pss]'] = None
            df_ODV.loc[i, 'SalinityQF'] = None
        if( df_ODV.loc[i, 'Temperature[°C]'] == -10000000000.0):
            df_ODV.loc[i, 'Temperature[°C]'] = None
            df_ODV.loc[i, 'TemperatureQF'] = None

        
    df_Nit = df_ODV[df_ODV['Nitrate[µmol/kg]'].notna()]
    # used the following for loop instead of ^.notna()]^ apprach to allow myself to retain 
    # the 'None'values inbetween profiles or 'Station' changes. So that the scatter plots 
    # trace correctly. This is a temp solution. I should probably split by station in the 
    # future but I'm slightly concerned about processing time when regraphing. 
    
        #for i in df_Nit.index[:-1]: 
        #    if( (df_Nit.loc[i, 'Station'] == df_Nit.loc[i+1, 'Station']) and 
        #        (df_Nit.loc[i, 'Nitrate[µmol/kg]'] == None) ):
        #        df_Nit.drop([i])
        #print(df_Nit)
    
    df_Nit['NitrateQF'] = df_Nit['NitrateQF'].apply(str)
    df_Nit['OxygenQF']  = df_Nit['OxygenQF'].apply(str)
    df_Nit['BBPQF']     = df_Nit['BBPQF'].apply(str)
    df_Nit['CHLAQF']    = df_Nit['CHLAQF'].apply(str)
    df_Nit['pH_QF']    = df_Nit['pH_QF'].apply(str)
    df_Nit['SalinityQF']    = df_Nit['SalinityQF'].apply(str)
    df_Nit['TemperatureQF']    = df_Nit['TemperatureQF'].apply(str)

    df_Nit = df_Nit.reset_index(drop=True)
    
    return df_Nit, nit_break_points#.groupby("Station", as_index=False) #removed list(kjdfjsdf)

In [42]:
def load_Bnc( fileName, param_List):
    """
    Example function call: 
        df = load_sprof("/Users/madison.soden/Downloads/2904010_Sprof.nc", ["DOXY"])
    """
    #loading relevant sprof into pandas array
    ds = xr.open_dataset(fileName, drop_variables=["DATA_TYPE", "FORMAT_VERSION", "HANDBOOK_VERSION", "HISTORY_SOFTWARE", "HISTORY_SOFTWARE_RELEASE", 
                                                   "HISTORY_REFERENCE", "HISTORY_DATE", "REFERENCE_DATE_TIME", "DATE_CREATION", "DATE_UPDATE", "HISTORY_ACTION",
                                                   "HISTORY_QCTEST", "HISTORY_START_PRES", "HISTORY_STOP_PRES", "PI_NAME", "PROJECT_NAME", "SCIENTIFIC_CALIB_COEFFICIENT"
                                                   "SCIENTIFIC_CALIB_EQUATION", 'PLATFORM_TYPE', 'FLOAT_SERIAL_NO', 'FIRMWARE_VERSION', 'WMO_INST_TYPE', 'JULD', 
                                                   'JULD_QC', 'JULD_LOCATION', 'LATITUDE', 'LONGITUDE', 'POSITION_QC', 'POSITIONING_SYSTEM', 'VERTICAL_SAMPLING_SCHEME',
                                                   'DIRECTION', 'DATA_CENTRE', 'DC_REFERENCE', 'SCIENTIFIC_CALIB_EQUATION', 'SCIENTIFIC_CALIB_COEFFICIENT', 'SCIENTIFIC_CALIB_COMMENT', 
                                                   'SCIENTIFIC_CALIB_DATE', 'HISTORY_INSTITUTION', 'HISTORY_STEP', 'HISTORY_PARAMETER', 'HISTORY_PREVIOUS_VALUE'])

    
    keep_column_list = ['N_PROF', 'N_LEVELS', 'PRES', 'PRES_ADJUSTED']
    for p in param_List:
        keep_column_list.append(p)
        keep_column_list.append(p+"_QC")
        keep_column_list.append(p+"_ADJUSTED")
        keep_column_list.append(p+"_ADJUSTED_QC")
        keep_column_list.append(p+"_ADJUSTED_ERROR")
    
    # convert to pandas df
    df = ds.to_dataframe().reset_index()
        #print(df.columns)
    
    df_Bnc = df.reindex(columns= keep_column_list)
    #df = df.reindex(columns=df.columns.union(["PRES_ADJUSTED", "PRES_ADJUSTED_QC", ...]))
        #print(df_sprof.head(25))
    df_Bnc = df_Bnc.loc[df_Bnc["DOXY"].notna()]
    df_Bnc = df_Bnc.drop_duplicates()
    
    return df_Bnc

In [43]:
def get_df_map_Bnc( dir_path, param_List): 
    file_paths = glob.glob(dir_path +"/BD*.nc")
    file_paths.sort()
    cycle_nums = [int(c[-6:-3]) for c in file_paths]
    Bnc_df_map = {c: load_Bnc(fp, param_List) for c, fp in zip(cycle_nums, file_paths)}

    return cycle_nums, Bnc_df_map

In [51]:
def make_df_dict(wmolist):
    float_dict = dict.fromkeys(wmolist)
    for wmo in wmolist:
        float_dict[wmo] = {
            "r_df": None,
            "r_df_grouped": None,
            "r_mask": None,
            "a_df": None,
            "a_df_grouped": None,
            "a_mask": None,
            "break_points": None,
            "profile_list": None, 
            "max_prof": None,
            "min_prof": None,
            "num_prof": None,
            "colorscale": None }
        float_dict[wmo]['r_df'], foo    = load_ODV( f'{matlab_data_dir}{wmo[:7]}/ODV{wmo}.TXT')
        float_dict[wmo]['r_df_grouped'] = float_dict[wmo]['r_df'].groupby("Station", as_index=False)
        float_dict[wmo]['a_df'], bpts   = load_ODV( f'{matlab_data_dir}{wmo[:7]}/QC/ODV{wmo}QC.TXT')
        float_dict[wmo]['a_df_grouped'] = float_dict[wmo]['a_df'].groupby("Station", as_index=False)
        float_dict[wmo]['break_points'] = bpts
        float_dict[wmo]['profile_list'] = float_dict[wmo]['r_df']['Station'].unique().tolist()
        float_dict[wmo]['max_prof']     = max( float_dict[wmo]['profile_list'])
        float_dict[wmo]['min_prof']     = min( float_dict[wmo]['profile_list'])
        float_dict[wmo]['num_prof']     = len( float_dict[wmo]['profile_list'])
        float_dict[wmo]['r_colorscale'] = "greys"
        float_dict[wmo]['a_colorscale'] = wmo_to_colorscale[wmo]
        
    max_prof_range = 1
    for wmo in wmolist: 
        if float_dict[wmo]["max_prof"] > max_prof_range:
            max_prof_range = float_dict[wmo]["max_prof"]

    return float_dict, max_prof_range

### load data from list of WMO#s
- functionality for SProfs only right now

In [52]:
float_df_dict, max_prof_range = make_df_dict(WMO_LIST)

### MISC. Sprof and BR/D import testing

In [ ]:
f_22_nprof = len(df_22_grouped)+1   #will return 49 for a float w/ 49 profiles
f_24_nprof = len(df_24_grouped)+1   
f_25_nprof = len(df_25_grouped)+1   
f_9_nprof  = len(df_9_grouped)+1    

f_10_nprof = len(df_10_grouped)+1
f_11_nprof = len(df_11_grouped)+1 
f_2_nprof  = len(df_2_grouped)+1
f_4_nprof  = len(df_4_grouped)+1

max_nprof = max(f_4_nprof, f_2_nprof)

In [ ]:
df_sprof_22         = load_sprof( f_10_sprof, ['DOXY','NITRATE','PH_IN_SITU_TOTAL','CHLA','BBP700'])
df_sprof_22_grouped = df_sprof_22.groupby("N_PROF", as_index=False) 

In [ ]:
df_sprof_24         = load_sprof( f_24_sprof, ['DOXY','NITRATE','PH_IN_SITU_TOTAL','CHLA','BBP700'])
df_sprof_24_grouped = df_sprof_24.groupby("N_PROF", as_index=False)

In [ ]:
df_sprof_25         = load_sprof( f_25_sprof, ['DOXY','NITRATE','PH_IN_SITU_TOTAL','CHLA','BBP700'])
df_sprof_25_grouped = df_sprof_25.groupby("N_PROF", as_index=False)

In [ ]:
df_sprof_9         = load_sprof( f_9_sprof, ['DOXY','NITRATE','PH_IN_SITU_TOTAL','CHLA','BBP700'])
df_sprof_9_grouped = df_sprof_9.groupby("N_PROF", as_index=False)

In [ ]:
df_sprof_10 = load_sprof( f_10_sprof, ['DOXY','NITRATE','PH_IN_SITU_TOTAL','CHLA','BBP700'])
df_sprof_10_grouped = df_sprof_10.groupby("N_PROF", as_index=False)

In [ ]:
df_sprof_11 = load_sprof( f_11_sprof, ['DOXY','NITRATE','PH_IN_SITU_TOTAL','CHLA','BBP700'])
df_sprof_11_grouped = df_sprof_11.groupby("N_PROF", as_index=False)

In [ ]:
df_sprof_2 = load_sprof( f_2_sprof, ['DOXY','NITRATE','PH_IN_SITU_TOTAL','CHLA','BBP700'])
df_sprof_2_grouped = df_sprof_2.groupby("N_PROF", as_index=False)

In [ ]:
df_sprof_4 = load_sprof( f_4_sprof, ['DOXY','NITRATE','PH_IN_SITU_TOTAL','CHLA','BBP700'])
df_sprof_4_grouped = df_sprof_4.groupby("N_PROF", as_index=False)

### Run Data Dashboard
  (will open in another tab)

In [53]:
app = Dash(__name__)
#buffer = io.StringIO()
#fig.write_html(buffer)
#html_bytes = buffer.getvalue().encode()
#encoded = b64encode(html_bytes).decode()

app.layout = html.Div([
    html.Div([
        html.H5("Mode: "),
            dcc.Checklist(
                options = [
                     {'label': 'Real',      'value': 'r'},
                     {'label': 'Adj/Delay', 'value': 'a'},
                ],
                value = [],
                inline= True,
                id='data_mode',), 
        html.H5("Floats: "),
            dcc.Checklist(
                options = [ {'label': f"WMO {wmo}", 'value': wmo} for wmo in float_df_dict.keys()],
                #options = [
                #     {'label': 'WMO 2904010', 'value': '2904010'},
                #     {'label': 'WMO 2904011', 'value': '2904011'},
                #],
                value = [],
                inline= True,
                id='wmo_list',),
        html.H5("Variable"),
            dcc.Dropdown(
                id="n_param",
                options=['Temperature[°C]', 'Salinity[pss]', 'Oxygen[µmol/kg]', 'Nitrate[µmol/kg]', 'pH_Insitu_total', 'BBP_700[1/m]', 'CHLA[mg/m^3]'],
                value='Nitrate[µmol/kg]',
            ),    
        html.H5("Filter by Cycle #:"),
            dcc.RangeSlider(
                id='range-slider',
                min=1, max=max_prof_range, step=1,
                value=[0, max_prof_range],
                tooltip={
                "always_visible": True,
                "template": "{value}"
                },
            ), 
        
    ], style={'width': '49%', 'display': 'inline-block'}),
    html.Div([
        dcc.Graph(id="scatter-plot")
        ], style={'height': '70%'}) #'display': 'inline-block'
    ])

@app.callback( 
    Output("scatter-plot", "figure"), 
    Input( "range-slider", "value"),
    Input( "n_param",      "value"),
    Input( "wmo_list",     "value"),
    Input( "data_mode",     "value") )

def update_plot(slider_range, n_param, wmo_list, data_mode):
 
    low, high = slider_range
    for wmo in wmo_list:
        for mode in data_mode:
            float_df_dict[wmo][f"{mode}_df"][f"{mode}_mask"] = (float_df_dict[wmo][f"{mode}_df"]['Station'] >= low) & (float_df_dict[wmo][f"{mode}_df"]['Station'] <= high)
        

    if(n_param=='Nitrate[µmol/kg]'):  
        start_x_range= [0,35]
        color_str = 'NitrateQF'
    elif(n_param=='Oxygen[µmol/kg]'):
        start_x_range= [100,250]
        color_str = 'OxygenQF'
    elif(n_param=='pH_Insitu_total'):
        start_x_range= [7.6,8.1]
        color_str = 'pH_QF'
    elif(n_param=='CHLA[mg/m^3]'):
        start_x_range= [0, 0.25] 
        color_str = 'CHLAQF'
    elif(n_param=='BBP_700[1/m]'):
        start_x_range= [0, 0.004]
        color_str = 'BBPQF'
    elif(n_param=='Temperature[°C]'):
        start_x_range= [0, 35]
        color_str = 'TemperatureQF'
    elif(n_param=='Salinity[pss]'):
        start_x_range= [34, 38]
        color_str = 'SalinityQF'
        
    fig = go.Figure()

    
    ##float df dict implmentation## 
    for wmo in wmo_list:#float_df_dict.keys(): 
        for mode in data_mode:
            float_colors_list = get_color_scale_rgb( float_df_dict[wmo][f'{mode}_colorscale'] , high+1 - low)
            if 'a' not in data_mode: float_colors_list = get_color_scale_rgb( float_df_dict[wmo][f'a_colorscale'] , high+1 - low)
            mode_color= float_colors_list[-10:]
            if mode=='r': mode_color= float_colors_list[0:10]
            trace_legend_bool = True
            #inital fig made with plotly express, very important for html color toggles
            fig.add_trace( px.scatter( float_df_dict[wmo][f"{mode}_df"][  float_df_dict[wmo][f"{mode}_df"][f"{mode}_mask"]  ], 
                              x=n_param, y='Depth[m]', color=color_str, color_discrete_sequence=mode_color,
                              hover_data=['Station'], symbol=color_str, 
                              range_x = start_x_range).update_traces(legendgroup= f"{wmo}_QC").data[0])
            fig.update_traces(marker=dict(size=3))
    
       
            #looping profiles to add as traces
            i=0 #color counter
            for profile_num, df in float_df_dict[wmo][f"{mode}_df_grouped"]: 
                if(profile_num in range(low, high+1)): #+1 added to have range inclusive of 'high' #range(high, low-1, -1)): #
                    fig.add_trace(
                        go.Scatter(
                            y= df['Depth[m]'], 
                            x= df[n_param],
                            name= f"{mode}_{wmo} {str(profile_num)}",
                            hoverinfo='skip',
                            legendgroup= f"{wmo}_{mode}",
                            line= dict(color= float_colors_list[i], width= 1), #line_colors_reds
                            showlegend= trace_legend_bool) )
                    #print(f"{i}: {float_colors_list[i]}")
                    i+=1 #increment color counter
                    trace_legend_bool = False #turn off legend lables after first profiles trace
                
      
    ##Formating##
    fig.update_layout(    
        yaxis= dict(
            domain= [0.05, 1],
            ticksuffix= " m",
            title=dict(text="Depth", font=dict(size=14,color ="#007FBF")),
            tickfont= dict(color= "#007FBF", size= 10),
            ticks="inside", 
            tickwidth=1,
            ticklen=10,
            tickcolor="#007FBF",
            autorange= "reversed",
            showgrid=True,
            fixedrange=False,
            zeroline= False
        ),
        xaxis= dict(                         
            title=dict( text=n_param, font=dict(size=14,color ="#007FBF")),
            tickfont= dict( color= "#007FBF", size= 10),
            showgrid=True,
            tickwidth=1,
            side= "top",
            ticks="inside", 
            tickcolor="#007FBF", 
            ticklen=10,
            fixedrange=False,
            zeroline= False 
        ),
        legend= dict( xanchor= "right", yanchor= "bottom", y= 0.1, x= 1.25,
                      font = dict( size = 8),
                      bgcolor="#F2F5F7",),
        #title_text= "<b>NIT_ADJUSTED</b>",
        plot_bgcolor = "#F2F5F7", #"#E6F7FF", # "#0085ca"
        #title_font_family= "Open Sans",
        title_font_size= 16,
        title_font_color= "#003087",
        title_x= 0.01,
        title_y= .99,
        width= 700,
        height= 700)
    
    #HTML export
    fig.write_html('testing1234.html')
    
    return fig

app.run_server(jupyter_mode='tab', debug=True)

Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>

In [30]:
foo = "6999992"
print(foo[:7])

6999992


### Useful lines 
```
 - fig.gettraces()
 - print(df.columns.tolist())
 - preview_df(df_10)
 - ds_2 = xr.open_dataset( f_2_sprof)
   print(ds_2)

### Miscellaneous Scrap Code 
please ignore

In [ ]:
def gen_Netcdf( floatNum, BRfileName, BDfileName):
    #Copying BRnetcdf data object to new working netcdf file
    os.system('cp ' + data_dir + floatNum+ '/' +  BRfileName + " " + data_dir + floatNum+ '/' + 'BD/'+BDfileName)
        ##Unix/Mac format os.system('cp file1.txt file7.txt')
        ##Windows format os.system('copy file1.txt file7.txt')
    
    #Opening working BDfile 
    ##format to open file in NETCDF3_CLASSIC
    ## 'a' (append) mode automatically reads previous netcdf format
    BDfile = netCDF4.Dataset(data_dir + floatNum+ '/' + 'BD/' + BDfileName, 'a') #opening BDfile
    return BDfile

In [ ]:
def set_DOXY_vars( BDfile, df_ODV, OxygenQC_index, sageGain):
    #setting DOXY_ADJUSTED, DOXY_ADJUSTED_QC, and DOXY_ADJUSTED_ERROR
    #initalizing empty arrays to write ODV data too
    
    DOXY_Adjusted_Array = np.ma.empty(shape = (BDfile.dimensions["N_LEVELS"].size), fill_value=99999.0, dtype='float32')
    DOXY_Adjusted_Array[:] = 99999.0
    DOXY_Adjusted_Array.mask = True
    
    DOXY_AdjustedQC_Array = np.ma.empty(shape = (BDfile.dimensions["N_LEVELS"].size), dtype='|S1')
    DOXY_AdjustedQC_Array[:] = ''
    DOXY_AdjustedQC_Array.mask = True

    DOXY_Adjusted_Error = np.ma.empty(shape = (BDfile.dimensions["N_LEVELS"].size), fill_value=99999.0, dtype='float32')
    DOXY_Adjusted_Error[:] = 99999.0
    DOXY_Adjusted_Error.mask = True
    
    
    for i in range(BDfile.dimensions["N_LEVELS"].size):
        #DOXY_Adjusted_Array
        x = np.float32( BDfile.variables["DOXY"][1, i] *np.float32(sageGain))
        if ~BDfile.variables["DOXY"][1,i].mask: 
            DOXY_Adjusted_Array[i] = x 
            #set DOXY_AdjustedQC_Array to 'Good'. or 1
            DOXY_AdjustedQC_Array[i] = 1

        #iterating through ODV file for selected cycle
        for row in df_ODV.index.values:
            if (np.float32(df_ODV.iloc[row]['Pressure[dbar]']) == np.round( BDfile.variables['PRES'][1,i], 2)):
                if ~(BDfile.variables["DOXY"][1,i].mask): 
                    DOXY_Adjusted_Error[i] = clim_adjusted_error_calc( pd.to_datetime(df_ODV.iloc[row]['mon/day/yr'], format= "%m/%d/%Y" ).to_julian_date(), 
                                                                          pd.to_datetime(df_ODV.iloc[row]['mon/day/yr'], format= "%m/%d/%Y" ).to_julian_date(),
                                                                          df_ODV.iloc[row]['Temperature[°C]'], 
                                                                          df_ODV.iloc[row]['Salinity[pss]'])


In [ ]:
#foo_row = pd.DataFrame({'Station': [112], 'Cruise':None, 'Type':None, 'mon/day/yr':None, 'hh:mm':None, 'Lon [°E]':None, 'Lat [°N]':None, 'QF':None, 'Pressure[dbar]':None, 'PressureQF':None,
#                        'Temperature[°C]':None, 'TemperatureQF':None, 'Salinity[pss]':None, 'SalinityQF':None, 'Sigma_theta[kg/m^3]':None, 'QF.4':None, 'Depth[m]':None, 'QF.5':None, 
#                        'Oxygen[µmol/kg]':None, 'OxygenQF':None, 'OxygenSat[%]':None, 'QF.7':None, 'Nitrate[µmol/kg]':None, 'NitrateQF':None, 'CHLA[mg/m^3]':None, 'CHLAQF':None, 'BBP_700[1/m]':None,
#                        'BBPQF':None, 'CDOM[ppb]':None, 'QF.11':None, 'pH_Insitu_total':None, 'pH_QF':None, 'CP660[1/m]':None, 'QF.13':None})
#df_22 = pd.concat([df, new_row], ignore_index=True)
#df_22.loc[len(df_22)] = [None] * len(df_22.columns)
#df_22.loc[len(df_22)-1, 'Station'] = 112
#print(df_22)